In [1]:
import torch
import torchvision
from imagenet import ImageNet
import os
import scipy

dataroot = '/userdata/smetzger/data/'

transform_train = None
transform_test = None


imnet = torchvision.datasets.ImageNet(dataroot + 'imagenet/imagenet12', download=True)

You set download=True, but a folder 'train' already exist in the root directory. If you want to re-download or re-extract the archive, delete the folder.


In [2]:
idx120 = [16, 23, 52, 57, 76, 93, 95, 96, 99, 121, 122, 128, 148, 172, 181, 189, 202, 210, 232, 238, 257, 258, 259, 277, 283, 289, 295, 304, 307, 318, 322, 331, 337, 338, 345, 350, 361, 375, 376, 381, 388, 399, 401, 408, 424, 431, 432, 440, 447, 462, 464, 472, 483, 497, 506, 512, 530, 541, 553, 554, 557, 564, 570, 584, 612, 614, 619, 626, 631, 632, 650, 657, 658, 660, 674, 675, 680, 682, 691, 695, 699, 711, 734, 736, 741, 754, 757, 764, 769, 770, 780, 781, 787, 797, 799, 811, 822, 829, 830, 835, 837, 842, 843, 845, 873, 883, 897, 900, 902, 905, 913, 920, 925, 937, 938, 940, 941, 944, 949, 959]
total_trainset = ImageNet(root=os.path.join(dataroot, 'imagenet/imagenet12'), transform=transform_train)
testset = ImageNet(root=os.path.join(dataroot, 'imagenet/imagenet12'), split='val', transform=transform_test)

In [3]:
total_trainset = ImageNet(root=dataroot + 'imagenet/imagenet12', transform=None) # TODO for LINCLS, make this train and test xforms.
 # TODO CHANGE ROOT TO args.data

In [7]:
import numpy as np

train_idx = np.arange(len(total_trainset))
print(len(train_idx))

np.random.seed(1337) #fingers crossed. 
np.random.shuffle(train_idx)

train_idx = train_idx[:50000]

kfold = 0
subset = np.arange(kfold*10000, (kfold+1)*10000)
print(train_idx[:10])
valid_idx = train_idx[subset]
train_idx = np.delete(train_idx, subset)
print(train_idx[:10])
print(len(valid_idx))
print(len(train_idx))

for i in train_idx: 
    if i in valid_idx: 
        print('bollox')


train_dataset = total_trainset
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetSampler(valid_idx)
train_sampler = torch.utils.data.distributed.DistributedSampler(Subset(train_dataset, train_idx))

print('train_sampler', train_sampler)

1281167
[1235085 1212951  925883  987848  662747 1258323  191503  724041  668259
 1217023]
[ 550553  522441 1088721 1208452   90740  168075  855441 1187826  535299
  367904]
10000
40000


NameError: name 'SubsetRandomSampler' is not defined

In [19]:
class SubsetSampler(Sampler):
    """Samples elements from a given list of indices, without replacement.
    Arguments:
        indices (sequence): a sequence of indices
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (i for i in self.indices)

    def __len__(self):
        return len(self.indices)


train_sampler = None

split = .15
split_idx = 1 # WHICH KFOLD WE ARE USING. 
target_lb = -1
multinode = False

if split > 0.0:
    sss = StratifiedShuffleSplit(n_splits=5, test_size=split, random_state=0)
    sss = sss.split(list(range(len(total_trainset))), total_trainset.targets)
    for _ in range(split_idx + 1):
        train_idx, valid_idx = next(sss)

    if target_lb >= 0:
        train_idx = [i for i in train_idx if total_trainset.targets[i] == target_lb]
        valid_idx = [i for i in valid_idx if total_trainset.targets[i] == target_lb]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetSampler(valid_idx)

    if multinode:
        train_sampler = torch.utils.data.distributed.DistributedSampler(Subset(total_trainset, train_idx), num_replicas=dist.get_world_size(), rank=dist.get_rank())
else:
    valid_sampler = SubsetSampler([])

    if multinode:
        train_sampler = torch.utils.data.distributed.DistributedSampler(total_trainset, num_replicas=dist.get_world_size(), rank=dist.get_rank())
       

In [18]:
print(train_idx[:10])

[   9810  717988   42759  399521  949984  369814  516438   40877 1005088
  979428]
